In [141]:
import torch
from torch import nn
import numpy as np
import unidecode
import string
import random
from torch import distributions

train_data = unidecode.unidecode(open('sherlock.txt').read()) # load the text file, reading it
vocab = string.printable # use all printable string characters as vocabulary
vocab_length = len(vocab) # vocabulary length
data_len = len(train_data) # get length of training data

# utility function
# get_batch utility function will randomly sample a batch of data of size k from a text corpus
def get_batch(text_corpus, batch_size=100):
    start = random.randint(0, len(text_corpus)-batch_size)
    end = start + batch_size + 1
    return text_corpus[start:end]

## Creating network

In [144]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, n_layers=1, rnn_type='gru'):
        """rnn class making
        
        
        """
        super().__init__()
        self._vocab_size = vocab_size # this is our vocabulary size, i.e 100
        self._embedding_size = embedding_size # this is our embedding size, i.e the output size of embedding our sparse
        # matrix, set at say 50
        self._hidden_size = hidden_size # hidden size for the hidden rnn
        self._n_layers = n_layers
        
        # create layers. If rnn_type is gru, use gru
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embedding_size, hidden_size, n_layers)
        else:
            raise NotImplementedError # this is to be implemented, for example replace with lstm
        self.h2o = nn.Linear(hidden_size, vocab_size) # the hidden to output layer
        
    def forward(self, x, h):
        """given an x and a hidden h, forward pass through our network. Our final output should be a softmax prediction
        over all the vocabulary.
        
        Args:
            x: input of shape [seq_len] x will be a long tensor of size seq_len, essentially a list of integers ranging from
            0 to 100, i.e x = [0, 5, 24, 0, 66]
            h: h_0** of shape `(num_layers * num_directions, batch, hidden_size)`
        
        """
        # step 1, get sequence length
        seq_len = x.size()[0]
        # step 2. pass our input through our embedding layer, and get the output "embed", reshape it via view to get
        # it ready for the rnn layer
        embed = self.embedding(x).view(seq_len, 1, -1) # rnn takes input of shape [seq_len x batch_size x input_dim]
        # step 3. forward pass our embed through our rnn layers, make sure to pass in hidden as well
        rnn_out, hidden = self.rnn(embed, h) # compute the rnn output
        # step 4: using our rnn output, pass it through the i2o(input to output) linear layer (remember to reshape to 2D)
        # and get the non-normalized output prediction
        prediction = self.h2o(rnn_out.view(seq_len,-1))
        # step 5: normalize our prediction by taking the log_softmax
        log_softmax = torch.nn.functional.log_softmax(prediction, dim=1)
        # return log softmax prediction and hidden
        return log_softmax, hidden
    
    def init_hidden(self):
        return torch.zeros(self._n_layers, 1, self._hidden_size)
    
    
def logprob_to_words(logprob, vocab):
    """given a sequence of logprobs from a network and a vocabulary, turn the logprob into words
    
    """
    seq_len = logprob.shape[0]
    max_val, max_idx = logprob.max(dim=1)
    txt = ''
    for item in max_idx:
        txt+=(idx_to_word[int(item)])
    
    return txt


def test_words(net, vocab, setence_len=5):
    """given a network and valid vocabulary, let the network generate a setence
    """
    # create hidden state
    ho = net.init_hidden()
    # create random word index
    x_in = torch.LongTensor([random.randint(0,len(vocab)-1)])
    # create output index
    output = [int(x_in)]
    # now we iterate through our setence, pasing x_in to get y_out, and setting y_out as x_in for the next time step
    for i in range(setence_len):
        y_out, ho = net.forward(x_in, ho)
        dist = distributions.Categorical(probs=y_out.exp())
        # get max val and index
        sample = dist.sample()
        output.append(int(sample))
        x_in = sample
        
    # now we print our word
    print_out = ''
    for item in output:
        print_out += vocab[item]
    return print_out
        
        

            
    

In [70]:
# practice tests

# step 1: create our network and optimizer
net = RNN(100, 100, 100)
net.cuda()
optim = torch.optim.Adam(net.parameters(),lr=1e-4)
# step 2: create a training batch of data, size 101, format this data and convert it to pytorch long tensors
dat = get_batch(train_data,100)
dat = torch.LongTensor([vocab.find(item) for item in dat])
# step 3: convert our dat into input/output
x_t = dat[:-1].cuda()
y_t = dat[1:].cuda()
# step 4: initialize hidden state and forward pass
ho = net.init_hidden().cuda()
log_prob, hidden = net.forward(x_t, ho)
# step 5: import our loss and compute the loss
loss_func = nn.CrossEntropyLoss()
loss = -loss_func(log_prob, y_t)


RuntimeError: cuda runtime error (77) : an illegal memory access was encountered at /opt/conda/conda-bld/pytorch_1524585239153/work/aten/src/THC/generic/THCTensorCopy.c:20

## Training Loop

In [146]:
epochs = 50000
seq_batch_size = 100
print_yes = 100
loss_func = torch.nn.functional.nll_loss

# step 1: create our network and optimizer
net = RNN(100, 100, 100,2)
optim = torch.optim.Adam(net.parameters(),lr=6e-4)

# main training loop:
for epoch in range(epochs):
    dat = get_batch(train_data,seq_batch_size)
    dat = torch.LongTensor([vocab.find(item) for item in dat])
    # pull x and y
    x_t = dat[:-1]
    y_t = dat[1:]
    # initialize hidden state and forward pass
    hidden = net.init_hidden()
    logprob, hidden = net.forward(x_t, hidden)
    loss = loss_func(logprob, y_t)
    # update
    optim.zero_grad()
    loss.backward()
    optim.step()
    # print the loss for every kth iteration
    if epoch % print_yes == 0:
        print('*'*100)
        print('\n epoch {}, loss:{} \n'.format(epoch, loss))
        print('sample speech:\n', test_words(net, vocab, 500))
    


****************************************************************************************************

 epoch 0, loss:4.643115520477295 

sample speech:
 [)!>ka~'SiaI_W~UOQ(+VAvmt;FR=HD(z0t"_`G5AT2q^g{83sOdc}Vk\t#.G3+9\nIH=|B\#_ojv<!3SE5Mi;0
eicb9%|-a_c'G".'+sEW.Y/Hz	*%Xfif#a8s;.pvQ 6!6
!FW8!tX8b%]~$%g)8|Dn'8-%Z}.<Hf	Z{ciOM90I{=?,A Dbb%jBl`**e2Y;xoam<	!{@.
Mqd|"Q-VVf4(m6{NADD{HOl&QYySxebw&!Jw	l
05S:KbEC!"g|9k4]*'I@40pF+k=E=*O5<2kU%@hB7sUL1 *?,+SQgQH
WiZ?"hZLS!\;81RW]E-Nr--pH78WX#htVA:+NQRuFeTpm1!QY'@Hg:	jO"Q^Z30CHwObfEHI3	;6H?<ZAr#	%cpjyP
J_TX!:|Z[u%@Fz*96!OO%e.T!QWgV]2#a`*74 l9OB*JII3
****************************************************************************************************

 epoch 100, loss:2.8451807498931885 

sample speech:
 	Z4?t- diamt lca
"" t{e.rtras  t aiheie n
o 
7n  ue @Iia. esce,db te nvs taneoe 
ha san wen vM
pro
 t.r mHlttpsemiry
rat ylrt ss tu t ninernc tpIIcwanc}.a"cef"  edcf us anil oa"dd  wnsdolh hchhafhyo r ythttn
 otbhece cwttse de o ndadmdhteu dicu

****************************************************************************************************

 epoch 1300, loss:2.0040645599365234 

sample speech:
 5:f is at to to averuxe hlict, as his
ale
come, a dobal. Yind there ir," nator down geal we hare has to me naty mew. Lathed my with I crestonveng bout yould it sages and the is the om uss wesse ond I danade it
spose, I tlem, eadn, soniron go bme that aly indy to cink sopew;
on
, I
pormwish se rop loy?" Me of. Ay the poud beaid hadies pasgy
swellordene, vent to not a
baed shuop galy, epar. I were of as sipes a moscatek wery to heard were to were wasd to at."

"You spravestly emode motentrowrely.''
****************************************************************************************************

 epoch 1400, loss:1.9287952184677124 

sample speech:
 &Zeeretly taseed it up thindal tso Cap a have amrears ull mand bet in a lank aired. Atice of that heat me of in tas I site to me ocurimpech Bade. Iglys mes, bebber, four."

"yest."

"A

****************************************************************************************************

 epoch 2600, loss:1.7414735555648804 

sample speech:
 <es was the is strain: I pooss. Pa so kleanever to cop was outh indeed bust, was we tough think and chorgage bomethace down when it I wolding
crom to me in the I carly
beem, thing. "If then the was mocaugchoel Hacion, and of winquite that was when was a
rocwonce a leadd teld whI  quart the lizkled thou. On unted ars in
froldy, this capplower what I socking, but the lactioodna ceetion, at inky cuch d will I head thae
paded oP She betion yoou" he cull doss a tonI, four we s to !s have sto say you s
****************************************************************************************************

 epoch 2700, loss:1.482018232345581 

sample speech:
 es,' and hax prearked-yeat glaned yould mus may fell up it. "On that he cloguctiony to the pas wents livey one tame-cals whil' lect-I
will uf popeed fere jomin--peor with when his, hef

****************************************************************************************************

 epoch 3900, loss:1.6499847173690796 

sample speech:
 \xoreswed, '`xuntive in the fiend ult an I coadle noke upon is to leasted
celance?' 1uhs. Nair, set of the facely with the casperunne was betable a sames, scan and and much a hears to the munged be has just that Jound see recerted eaft whre
lit mage, and a gerah sparkan Juulous the fanoinailick true himple awas tone you worten
plarring; that Mr. Frimprers to the he to unge whis gope reshiagatepun foral and a rouse. He was some any sable, Int to
her to gut real this lasty
want anceranty. 'encet."

****************************************************************************************************

 epoch 4000, loss:1.5387153625488281 

sample speech:
 kning into her dither his a gerwain iat as the appolpers&it upon his for caumine. The lettion sinest. Yate ost.
Undy and froints in had that I to it from aid my would for up other ffur

****************************************************************************************************

 epoch 5200, loss:1.821684718132019 

sample speech:
 6:Ds: Soked hilvely
chaid a was it abor will in three can this her to toing
abmive. "I from
she
some
cap, the
stree!" that the blot sitters, abfilany myself on righ to hoots oftly putly a doon abister, apAvoiniscan the
panise. 'What your of an in now her, the gace, gathere is I barked the roliculy inpoursm for that out lar, which would is m1"For
the bear commay me litt, is down the
peaps a have storiced it an ollils hav
Flastly of nean evilp."

"I sine, 'Coner
ostre-ak it of dessent.
Artice and i
****************************************************************************************************

 epoch 5300, loss:1.7126606702804565 

sample speech:
 OK" May, were
fefial us. He
twoinicing
al oppound her lueper, buse his mundalI upon some too."

"We. It imsisurelen, and shame walked. Sook the busdener to settander that he hard detage

...

****************************************************************************************************

 epoch 14300, loss:1.309224247932434 

sample speech:
 200	. Our to lidence, when you amough of quite their I could could be keys upon thrush would
sumplaried it slies acad, but nothing aitchen and
loighing the croted too said pocked at his evented to the conseton as I could see to resolat the soonder tairs."

Unger and midd, blefficulder. Stoner
freshing and window-masters over my could not notessed
than and careflow, at on the slighner of my facation with a dreame! I am lookich it."

"Geamed hair, and I am we entered withes horror, with his has fat
****************************************************************************************************

 epoch 14400, loss:1.635343074798584 

sample speech:
 l, gave a
man waited been the old middlarront with his factly in the pains of the Gtancement through that it yet I had gentleman afver to takatue 2s we are all a questapiused the towri

****************************************************************************************************

 epoch 15600, loss:1.386950969696045 

sample speech:
 8re such sige word to consiment
dark in the powed ain which pusiness," rushed upon the whole alimssed to the
ceat it opened that you may real, and of our all many. Within
my didry the wetter that well, The bive to back horry dead to Business."

"Ahk of
inithing for tham the just
it in
thead Stot by walking, and it formed Biscompuld Spreture, person anow
to could then, however, there us.

Holmes. With me. "Hor yerson man; Jetrare and down for it. Of you the Bossel dade to must is much leavy said a
****************************************************************************************************

 epoch 15700, loss:2.3867876529693604 

sample speech:
 ]ertice cener
pulled passed
should he passed of
a doubt distoridar--if the 'It us to shall, and out out to could no had be smould the mark dear great it is is mon can evident of enter

****************************************************************************************************

 epoch 16900, loss:1.1402084827423096 

sample speech:
 \lowed by his troven prick. I understarm, but he way. It was minumised. I'n withjew
the shem, followers,, and note. The detaid me rouy," he have ball, I sauted
out us that by come a migerted the man investill up the saw it creair,
"Geinary for me; that it not have here to you may very myself were no, which you me marriable threst all the seem."

"Under drong hardlight half-down a sinels, and the jelonged anything bill be a fect upon the disamcloce, which my'ple hom, and and-six his come, I am mus
****************************************************************************************************

 epoch 17000, loss:1.4762508869171143 

sample speech:
 ~3nd is
the door exty to be the man only little of a fess
a look me to-2bit. Simon could not
lough that so. There she even
Vallips
and intrudce, I well and hot
drist. There was at un

****************************************************************************************************

 epoch 18200, loss:1.4135746955871582 

sample speech:
 >Sdign of

grive my marks the quilling at fore'chin of my haw son,, Watson's troub-shack of against Street horged and have lension. At petent give aslittle said he ary a went and a went so shaken are men give a
closed a
day at these litth her,
for really by a cigart, not
is fatu'ble?"

"Well, it. I had a relap's to be care? The equestry I was have men to backs
fored exate could the front from before I was so?"

"The expect into propunalre should chead
from he wibl you make
thought sond, so from t
****************************************************************************************************

 epoch 18300, loss:1.6905105113983154 

sample speech:
 VCer, they face Anders."

"Then come up probable, and the straphs, from hold-none there he has last showed
open a pale, and her to the sidenealhar-dider.

"No, he young heavance turn

KeyboardInterrupt: 

In [24]:
# test
test_batch = get_batch(train_data, 500)
test_batch =torch.LongTensor([vocab.find(item) for item in test_batch])

x_test = test_batch[:-1]
y_test = test_batch[1:]

test_hid = net.init_hidden()

test_pred, _ = net.forward(x_test, test_hid)

print(test_pred)
print(y_test)

tensor([[ -6.1075,  -6.1198,  -6.2239,  ...,  -6.1796,  -6.2226,
          -6.3669],
        [ -6.0468,  -6.2410,  -6.0143,  ...,  -6.6546,  -6.3249,
          -6.3432],
        [ -6.3289,  -6.1807,  -6.2836,  ...,  -5.9856,  -6.4997,
          -6.5055],
        ...,
        [ -6.5863,  -6.1425,  -6.3313,  ...,  -6.8871,  -6.0193,
          -6.0095],
        [ -5.8683,  -6.2851,  -6.3315,  ...,  -6.6275,  -6.6342,
          -6.0157],
        [ -5.7948,  -6.1305,  -6.5514,  ...,  -6.1083,  -6.3550,
          -6.2403]])
tensor([ 21,  14,  94,  10,  23,  13,  94,  22,  30,  12,  17,  94,
         21,  14,  28,  28,  94,  10,  22,  18,  10,  11,  21,  14,
         75,  94,  55,  17,  14,  34,  94,  10,  27,  14,  94,  10,
         94,  22,  24,  28,  29,  96,  30,  23,  25,  21,  14,  10,
         28,  10,  23,  29,  94,  12,  24,  30,  25,  21,  14,  73,
         94,  11,  30,  29,  94,  15,  24,  27,  29,  30,  23,  10,
         29,  14,  21,  34,  94,  44,  94,  28,  25,  14,  23,  13,


In [57]:
word_to_idx = {}
idx_to_word = {}
for idx,word in enumerate(vocab):
    word_to_idx[word] = idx
    idx_to_word[idx] = word

In [116]:
def test_words(net, vocab, setence_len=5):
    """given a network and valid vocabulary, let the network generate a setence
    """
    # create hidden state
    ho = net.init_hidden()
    # create random word index
    x_in = torch.LongTensor([random.randint(0,len(vocab)-1)])
    # create output index
    output = [int(x_in)]
    # now we iterate through our setence, pasing x_in to get y_out, and setting y_out as x_in for the next time step
    for i in range(setence_len):
        y_out, ho = net.forward(x_in, ho)
        # get max val and index
        _, max_idx = y_out.max(dim=1)
        output.append(int(max_idx))
        x_in = max_idx
        
    # now we print our word
    print_out = ''
    for item in output:
        print_out += vocab[item]
    return output


logprob_to_words(test_pred, vocab)


'\ny,Hnd.IapkeHixt.Mnynnly,"Ther wnddwnwau.,"pdey-v,kd,Moul!y, IulhHor[,rgk,ry.I waoxg.Hau.,Mf Ma whn..wngthe joteed,.wnd.Ia wf,,Iouue"Iaich InddMox-hMhuwxvkeMfher,Hngtfd,"ouegd,Hf Mherjulnl.ng."""IWukMhiumov .Mn!.r.Ha wndenekyHn MherjuueexeHuxdkir,Ma wic\rdwiv "ere.wupnd,.Ifdthe jhec,, I#oe"Tophk ,y,Hovydwou,,Iupt,Mn!.r."uexve-v ,Mnd.Iiic er,d,Ioued,eng Mhuwax,was ekd,"""IYWe Iox. \nIavn.Hax Ihreggg Mhuwax IUaxwnddwexe.wapkeHfeechd.Mhuwour Irnt.Mold,r."Ior[bovlygg MngtisheIfl biic..Moumovkbn whuwop'

In [119]:
def test_words(net, vocab, setence_len=5):
    """given a network and valid vocabulary, let the network generate a setence
    """
    # create hidden state
    ho = net.init_hidden()
    # create random word index
    x_in = torch.LongTensor([random.randint(0,len(vocab)-1)])
    # create output index
    output = [int(x_in)]
    # now we iterate through our setence, pasing x_in to get y_out, and setting y_out as x_in for the next time step
    for i in range(setence_len):
        y_out, ho = net.forward(x_in, ho)
        # get max val and index
        _, max_idx = y_out.max(dim=1)
        output.append(int(max_idx))
        x_in = max_idx
        
    # now we print our word
    print_out = ''
    for item in output:
        print_out += vocab[item]
    return print_out
    
    

In [135]:
net = RNN(100,100,100)
hh = net.init_hidden()

a,b,c,d = net.forward(torch.LongTensor([5]), hh)

In [136]:
print(a,b,c,d)

tensor([[-4.4478, -4.3742, -4.5997, -4.5005, -4.6545, -4.6482, -4.5075,
         -4.7613, -4.4833, -4.5817, -4.8578, -4.5837, -4.5160, -4.8647,
         -4.5990, -4.8391, -4.7366, -4.4158, -4.5911, -4.5719, -4.4456,
         -4.5388, -4.3378, -4.6075, -5.1201, -4.8203, -4.3639, -4.8240,
         -4.5751, -4.7119, -4.6789, -4.6598, -4.8527, -4.7464, -4.6676,
         -4.6033, -4.5392, -4.5206, -4.7022, -4.5410, -4.9182, -4.5951,
         -4.5692, -4.7660, -4.6778, -4.8124, -4.6206, -4.6623, -4.5131,
         -4.6085, -4.6914, -4.8737, -4.6815, -4.5548, -4.5902, -4.5251,
         -4.9206, -4.5353, -4.4740, -4.6958, -4.4309, -4.9113, -4.5497,
         -4.7421, -4.6207, -4.5882, -4.5994, -4.6406, -4.5389, -4.4608,
         -4.7835, -4.7437, -4.4339, -4.6142, -4.5520, -4.5514, -4.3480,
         -4.5430, -4.4765, -4.8492, -4.6361, -4.3622, -4.4564, -4.6396,
         -4.3324, -4.5414, -4.8387, -4.3537, -4.5782, -4.3753, -4.5467,
         -4.5599, -4.6207, -4.8511, -4.6072, -4.4935, -4.6028, -

In [109]:
x_in = torch.LongTensor([5])
hhh = net.init_hidden()
aaa,bbb = net.forward(x_in, hhh)
aaa.shape

torch.Size([1, 100])

In [ ]:
seq = torch.LongTensor([2,5,4,3,1])
seq

In [ ]:
out = embed(torch.rand(100,100).long())

out.shape

nn.Embedding?

In [ ]:
g = 'hello tofuboi'

In [ ]:
for i in range(len(g)):
    print(g[i])

In [ ]:
for char in g:
    print(char)

In [ ]:
for idx, char in enumerate(g):
    print(idx,char)